# Individual CARMA R function tests
Compare function outputs with ```carma.py.functions.test.ipynb``` equivalent

In [1]:
library(Rcpp)
library(RcppArmadillo)
library(RcppGSL)
library(CARMA)
library(data.table)
library(dplyr)
library(magrittr)
library(devtools)
library(R.utils)
library(Matrix)
library(MASS)
library(glmnet)

Registered S3 methods overwritten by 'RcppGSL':
  method               from         
  predict.fastLm       RcppArmadillo
  print.fastLm         RcppArmadillo
  summary.fastLm       RcppArmadillo
  print.summary.fastLm RcppArmadillo


Attaching package: 'RcppGSL'


The following objects are masked from 'package:RcppArmadillo':

    fastLm, fastLmPure


The following object is masked from 'package:Rcpp':

    LdFlags



Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: usethis

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.25.0 (2022-06-12 02:20:02 UTC) successfully loaded. See ?R.oo for help.


Attachi

In [2]:
sumstats <- fread("/Users/hn9/Documents/GitHub/carmapy/tests/APOE_locus_sumstats.txt.gz")
ld <- fread("/Users/hn9/Documents/GitHub/carmapy/tests/APOE_locus_ld.txt.gz")
z.list <- as.list(sumstats$Z)
ld.list <- as.matrix(ld)

effect.size.prior <- "Normal"

set.seed(1)
n <- nrow(sumstats)
p <- length(z.list)
beta <- rep(0, p)
beta[1] <- 1
lambda.list <- list()
lambda.list[[1]] <- 1 / sqrt(p)

output.labels <- "."
label.list <- NULL
rho.index <- 0.99
BF.index <- 10
EM.dist <- "Logistic"
Max.Model.Dim <- 2e+5
all.iter <- 3
all.inner.iter <- 10
input.alpha <- 0
epsilon.threshold <- 1e-4
num.causal <- 10
y.var <- 1
tau <- 0.04
outlier.switch <- TRUE
outlier.BF.index <- 1 / 3.2
w.list <- NULL


In [3]:
log.2pi<-log(2*pi)
L<-length(z.list)
p.list<-list()
for(i in 1:L){
    z.list[[i]]<-as.matrix(z.list[[i]])
    p.list[[i]]<-nrow(z.list[[i]])
}
B<-Max.Model.Dim
all.B.list<-list()
for(i in 1:L){
all.B.list[[i]]<-list()
all.B.list[[i]][[1]]<-integer(0)
all.B.list[[i]][[2]]<-Matrix(nrow = 0,ncol=p.list[[i]],data=0,sparse = T)
}
q.list<-list()
if(!is.null(w.list)){
    for(i in 1:L){
    
    q.list[[i]]<-ncol(w.list[[i]])
    invariant.var.index<-which((apply(w.list[[i]][,-1],2,sd))==0)
    if(length(invariant.var.index)!=0){
        invariant.var<-w.list[[i]][,invariant.var.index+1]
        w.list[[i]]<-as.matrix(cbind(1,scale(w.list[[i]][,-1])))
        w.list[[i]][,invariant.var.index+1]<-invariant.var
    }else{
        w.list[[i]]<-as.matrix(cbind(1,scale(w.list[[i]][,-1])))
    }
    
    }
}
if(is.null(label.list)){
    for(i in 1:L){
    label.list[[i]]=paste0('locus.',i)
    }
}
Sigma.list<-list()
for(i in 1:L){
Sigma.list[[i]]<-as.matrix(ld.list[[i]])
}
S.list<-list()
for(i in 1:L){
S.list[[i]]<-integer(0)
}

all.C.list<-list()
for(i in 1:L){
    all.C.list[[i]]<-list()
    all.C.list[[i]][[1]]<-integer(0)
    all.C.list[[i]][[2]]<-Matrix(nrow = 0,ncol=p.list[[i]],data=0,sparse = T)
}

all.epsilon.threshold<-0
epsilon.list<-list()
for(i in 1:L){
    epsilon.list[[i]]<-epsilon.threshold*p.list[[i]]
    all.epsilon.threshold<-all.epsilon.threshold+  epsilon.threshold*p.list[[i]]
}
model.prior='Poisson'
standardize.model.space=T


In [7]:
EM.M.step.func<-function(input.response=NULL,w=w,input.alpha=0.5,EM.dist='Logistic'){
if(EM.dist=='Poisson'){
    count.index<-input.response
    cv.poisson<-cv.glmnet(w,count.index,family = 'poisson',alpha=input.alpha,type.measure='deviance' )
    cv.index<-which(cv.poisson$lambda==cv.poisson$lambda.min)
    glm.beta<-as.matrix(c(cv.poisson$glmnet.fit$a0[cv.index],cv.poisson$glmnet.fit$beta[-1,cv.index]))
}
if(EM.dist=='Logistic'){
    response.matrix<-matrix(c(1-input.response, input.response),length(input.response),2)
    cv.logistic<-cv.glmnet(x=w,y=response.matrix, family='binomial',alpha=input.alpha,type.measure = 'deviance')
    cv.index<-which(cv.logistic$lambda==cv.logistic$lambda.min)
    glm.beta<-as.matrix(c(cv.logistic$glmnet.fit$a0[cv.index],cv.logistic$glmnet.fit$beta[-1,cv.index]))
    
}

return(glm.beta=glm.beta)
}

# Predictor matrix w
w <- matrix(c(rep(1:10, 3), rep(11:20, 3)), nrow = 30)
# Response vector (Poisson distributed) (30 elements)
input.response <- c(rep(4:6, 10))
# Call the EM.M.step.func function
result_poisson <- EM.M.step.func(input.response = input.response, w = w, EM.dist = 'Poisson')
print(result_poisson)
pip <- c(0.95, 0.90, 0.75, 0.60, 0.50, 0.40, 0.30, 0.20, 0.15, 0.10)

# Example linkage disequilibrium (LD) matrix (10x10)
set.seed(42)
ld <- matrix(runif(100), nrow = 10)
ld <- ld %*% t(ld) # Making it positive semi-definite
diag(ld) <- 1 # Diagonal elements are 1

# Call the credible.set.fun.improved function
result <- credible.set.fun.improved(pip = pip, ld = ld)
print('Results with random but larger example inputs:')
print(result)


pip <- c(0.95, 0.90, 0.75, 0.60, 0.50, 0.40, 0.30, 0.20, 0.15, 0.10)

# Example linkage disequilibrium (LD) matrix (10x10)
ld <- matrix(c(
  1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.04, 0.03, 0.02,
  0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.04, 0.03,
  0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.04,
  0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05,
  0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1,
  0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2,
  0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3,
  0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4,
  0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5,
  0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1
), nrow = 10, byrow = TRUE)

# Call the credible.set.fun.improved function
result <- credible.set.fun.improved(pip = pip, ld = ld)
print('Results with exact example inputs:')
print(result)

       [,1]
s0 1.609438
   0.000000


In [16]:
credible.set.fun.improved<-function(pip,ld,true.beta=NULL,rho=0.99){
candidate.r<-c((seq(from=.5,0.95,by=0.05)),seq(0.96,0.99,0.01))

snp.list<-list()
colnames(ld)<-rownames(ld)<-1:nrow(ld)
for(r in 1:length(candidate.r)){
    working.ld<-ld
    cor.threshold<-candidate.r[r]
    pip.order<-order(pip,decreasing = T)
    snp.list[[r]]<-list()
    group.index<-1
    s<-1
    while(sum(pip[pip.order[s:length(pip.order)]])>rho){
    
    cor.group<-as.numeric(names(which(abs(working.ld[which(pip.order[s]==as.numeric(colnames(working.ld))),])>cor.threshold)))
    if(sum(pip[cor.group])>rho){
        group.pip<- pip[cor.group]
        snp.index<- cor.group[order(group.pip,decreasing = T)][1: min(which(cumsum( sort(group.pip,decreasing = T))>rho))]
        snp.list[[r]][[group.index]]<-snp.index
        group.index<-group.index+1
        pip.order<-pip.order[-match(snp.index,pip.order)]
        working.ld<-working.ld[-match(snp.index,as.numeric(colnames(working.ld))),-match(snp.index,as.numeric(colnames(working.ld)))]
    }else{
        s=s+1
    }
    }
    
}
if(sum(sapply(snp.list,length))!=0){
    group.index<-max(which(sapply(snp.list,length)==max(sapply(snp.list,length))))
    credible.set.list<-snp.list[[group.index]]
    if(!is.null(true.beta)){
    purity<-c()
    for(s in 1:length(credible.set.list)){
        purity<-c(purity, (mean(ld[ credible.set.list[[s]], credible.set.list[[s]]]^2)))
    }
    causal.snp<-ifelse(length(na.omit(match(unlist(credible.set.list),true.beta)))!=0,
                        length(na.omit(match(unlist(credible.set.list),true.beta))),0)
    length.credible<-length(credible.set.list)
    return(list(c(causal.snp,
                    ifelse(length.credible==0,NA,length.credible),
                    mean(sapply(credible.set.list,length)),
                    mean(purity)),credible.set.list))
    
    }else{
    purity<-c()
    for(s in 1:length(credible.set.list)){
        purity<-c(purity, (mean(ld[ credible.set.list[[s]], credible.set.list[[s]]]^2)))
    }
    length.credible<-length(credible.set.list)
    return(list(c(ifelse(length.credible==0,NA,length.credible),
                    mean(sapply(credible.set.list,length)),
                    mean(purity)),
                credible.set.list))
    }
}else{
    return(list(rep(0,4),list()))
}
}

pip <- c(0.95, 0.90, 0.75, 0.60, 0.50, 0.40, 0.30, 0.20, 0.15, 0.10)

# Example linkage disequilibrium (LD) matrix (10x10)
set.seed(42)
ld <- matrix(runif(100), nrow = 10)
ld <- ld %*% t(ld) # Making it positive semi-definite
diag(ld) <- 1 # Diagonal elements are 1

# Call the credible.set.fun.improved function
result <- credible.set.fun.improved(pip = pip, ld = ld)
print('Results with random but larger example inputs:')
print(result)


pip <- c(0.95, 0.90, 0.75, 0.60, 0.50, 0.40, 0.30, 0.20, 0.15, 0.10)

# Example linkage disequilibrium (LD) matrix (10x10)
ld <- matrix(c(
  1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.04, 0.03, 0.02,
  0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.04, 0.03,
  0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.04,
  0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05,
  0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2, 0.1,
  0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3, 0.2,
  0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4, 0.3,
  0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5, 0.4,
  0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 0.5,
  0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1
), nrow = 10, byrow = TRUE)

# Call the credible.set.fun.improved function
result <- credible.set.fun.improved(pip = pip, ld = ld)
print('Results with exact example inputs:')
print(result)

[1] "Results with random but larger example inputs:"
[[1]]
[1] 3.000000 2.333333 5.045620

[[2]]
[[2]][[1]]
[1] 1 2

[[2]][[2]]
[1] 3 4

[[2]][[3]]
[1] 5 6 7


[1] "Results with exact example inputs:"
[[1]]
[1] 0 0 0 0

[[2]]
list()



In [17]:
credible.model.fun<-function(likelihood,model.space,bayes.threshold=10){
na.index<-which(is.na(likelihood))
if(length(na.index)!=0){
    likelihood<-likelihood[-na.index]
    model.space<-model.space[-na.index,]
}
post.like.temp<-likelihood-likelihood[1]
post.prob<-exp(post.like.temp)/(sum(exp(post.like.temp)))
bayes.f<-post.prob[1]/post.prob
candidate.model<-1

credible.model.list<-list()
credible.model.list[[1]]<-list()
input.rs<-c()
for(ss in 1:length(which(bayes.f<bayes.threshold))){
    credible.model.list[[1]][[ss]]<-which(model.space[ss,]==1)
    input.rs<-c(input.rs,which(model.space[ss,]==1))
}
credible.model.list[[2]]<-data.frame(Posterior.Prob=post.prob[which(bayes.f<bayes.threshold)])
credible.model.list[[3]]<-unique(input.rs)
return(credible.model.list )


}

likelihood <- c(-150, -155, -152, -153, -154)

# Example model space: 5 models with 3 possible factors
model.space <- matrix(c(1, 0, 1,
                        0, 1, 1,
                        1, 1, 0,
                        0, 0, 1,
                        1, 1, 1), nrow=5, byrow=TRUE)

# Run the credible.model.fun function
result <- credible.model.fun(likelihood, model.space)

# Print the result
print(result)

[[1]]
[[1]][[1]]
[1] 1 3

[[1]][[2]]
[1] 2 3


[[2]]
  Posterior.Prob
1      0.8263261
2      0.1118311

[[3]]
[1] 1 3 2



# Testing module_cauchy_shotgun functions

In [18]:
input.prior.dist<-function(x){
            variable.index<-which(x==1)
            if(any(variable.index)){
            return( sum(posi.log.pro[variable.index])+sum(nega.log.pro[(1:p)[-variable.index]])-sum(nega.log.pro))
            }else{
            return(sum(nega.log.pro))
            }
        }
p <- 5 
posi.log.pro <- log(c(0.6, 0.5, 0.7, 0.4, 0.5))
nega.log.pro <- log(c(0.4, 0.5, 0.3, 0.6, 0.5))
x <- c(1, 0, 1, 0, 0)
result <- input.prior.dist(x)
print(result) 

[1] 1.252763


In [19]:
Poisson.prior.dist<-function(t){
        dim.model<-sum(t)
        result<-dim.model*log(lambda)+lfactorial(p-dim.model)-lfactorial(p)
        return(result)
    }
p <- 5
lambda <- 2.5
t <- c(1, 0, 1, 0, 0)

result <- Poisson.prior.dist(t)
print(result)

[1] -1.163151


In [21]:
beta.binomial.dist<-function(t){
    dim.model<-sum(t)
    result<- lbeta(dim.model+1,p-dim.model+9)-lbeta(1,p+9)
    return(result)
}
p <- 5
t <- c(1, 0, 1, 0, 0)

result <- beta.binomial.dist(t)
print(result)

[1] -4.356709


In [5]:
set.gamma.func<-function(input.S,condition.index=NULL){
    set.gamma.func.base<-function(S){
        add.function<-function(y){results<-(apply(as.matrix(S_sub),1,function(x){return(sort(c(x,y)))}))
        return(t(results))
        }
        
        set.gamma<-list()
        for(i in 1:3){
        set.gamma[[i]]<-c()
        }
        
        #set of gamma-
        if(length(S)==0){
        S_sub<-(1:p)
        set.gamma[[1]]<-c()
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(c(x,S))})
        set.gamma[[2]]<-as.matrix(set.gamma[[2]])
        set.gamma[[3]]<-c()
        }
        if(length(S)>1){
        S_sub<-(1:p)[-S]
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        if(length(S)>2){
            set.gamma[[1]]<-t(apply(as.matrix(set.gamma[[1]]),1,sort))
        }
        #set of gamma+
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-t(set.gamma[[2]])
        #set of gamma=
        set.gamma[[3]]<-add.function(set.gamma[[1]][1,])
        for(i in 2:nrow(set.gamma[[1]])){
            set.gamma[[3]]<-rbind(set.gamma[[3]],add.function(set.gamma[[1]][i,]))  
        }
        }
        if(length(S)==1){
        S_sub<-(1:p)[-S]
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-t(set.gamma[[2]])
        set.gamma[[3]]<-t(add.function(set.gamma[[1]][1,]))
        }
        return(set.gamma)
    }
    set.gamma.func.conditional<-function(input.S,condition.index){
        add.function<-function(y){results<-(apply(as.matrix(S_sub),1,function(x){return(sort(c(x,y)))}))
        return(t(results))
        }
        
        set.gamma<-list()
        for(i in 1:3){
        set.gamma[[i]]<-c()
        }
        S=input.S[-match(condition.index,input.S)]
        
        #set of gamma-
        if(length(S)==0){
        S=integer(0)
        S_sub<-(1:p)[-condition.index]
        set.gamma[[1]]<-c()
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(c(x,S))})
        set.gamma[[2]]<-as.matrix(set.gamma[[2]])
        set.gamma[[3]]<-c()
        }
        if(length(S)==1){
        S_sub<-(1:p)[-input.S]
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-as.matrix((t(set.gamma[[2]])))
        set.gamma[[3]]<-t(add.function(set.gamma[[1]][1,]))
        }
        if(length(S)>1){
        S_sub<-(1:p)[-input.S]
        if(length(S)>2){
            set.gamma[[1]]<-t(combn(S,length(S)-1))
            set.gamma[[1]]<-t(apply(as.matrix(set.gamma[[1]]),1,sort))
        }else{
            set.gamma[[1]]<-t(combn(S,length(S)-1))
        }
        set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
        set.gamma[[2]]<-as.matrix(t(set.gamma[[2]]))
        set.gamma[[3]]<-add.function(set.gamma[[1]][1,])
        factorial(3)
        for(i in 2:nrow(set.gamma[[1]])){
            set.gamma[[3]]<-rbind(set.gamma[[3]],add.function(set.gamma[[1]][i,]))  
        }
        }
        
        return(set.gamma)
    }
    
    if(is.null(condition.index)){
    results<-set.gamma.func.base(input.S)
    }else{
    results<-set.gamma.func.conditional(input.S,condition.index)
    }
    return(results)
}

# Define the value of p
p <- 5

# Example usage with input.S
input.S <- c(1,2,3)
result <- set.gamma.func(input.S)
print(result)

# Example usage with input.S and condition.index
input.S <- c(1,2,3)
condition.index <- 2
result_with_condition <- set.gamma.func(input.S, condition.index)
print(result_with_condition)

[[1]]
     [,1] [,2]
[1,]    1    2
[2,]    1    3
[3,]    2    3

[[2]]
     [,1] [,2] [,3] [,4]
[1,]    1    2    3    4
[2,]    1    2    3    5

[[3]]
     [,1] [,2] [,3]
[1,]    1    2    4
[2,]    1    2    5
[3,]    1    3    4
[4,]    1    3    5
[5,]    2    3    4
[6,]    2    3    5

[[1]]
     [,1]
[1,]    1
[2,]    3

[[2]]
     [,1] [,2] [,3]
[1,]    1    3    4
[2,]    1    3    5

[[3]]
     [,1] [,2]
[1,]    1    4
[2,]    1    5
[3,]    3    4
[4,]    3    5



In [9]:
sumstats <- fread("/Users/hn9/Documents/GitHub/carmapy/tests/APOE_locus_sumstats.txt.gz")
ld <- fread("/Users/hn9/Documents/GitHub/carmapy/tests/APOE_locus_ld.txt.gz")
z.list <- as.list(sumstats$Z)

p <- length(z.list)

# Example usage with input.S
input.S <- c(1:length(z.list))
result <- set.gamma.func(input.S)
print('no conditions')
print(result)

# Example usage with input.S and condition.index
input.S <- c(1:length(z.list))
condition.index <- 2
result_with_condition <- set.gamma.func(input.S, condition.index)
print('conditions')
print(result_with_condition)

[1] "no conditions"
[[1]]
       [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
  [1,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [2,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [3,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [4,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [5,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [6,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [7,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [8,]    1    2    3    4    5    6    7    8    9    10    11    12    13
  [9,]    1    2    3    4    5    6    7    8    9    10    11    12    13
 [10,]    1    2    3    4    5    6    7    8    9    10    11    12    13
 [11,]    1    2    3    4    5    6    7    8    9    10    11    12    13
 [12,]    1    2    3    4    5    6    7    8    9    10    1

In [6]:
S_sub <- matrix(c(1, 2, 3, 4, 5, 6), nrow=2)
y <- 7

add.function <- function(y) {
  results <- apply(as.matrix(S_sub), 1, function(x) {
    return(sort(c(x, y)))
  })
  return(t(results))
}

result <- add.function(y)
print(result)
class(result)

     [,1] [,2] [,3] [,4]
[1,]    1    3    5    7
[2,]    2    4    6    7


[1] "matrix" "array"

In [7]:
# Define the value of p
p <- 5
input_S <- c(1,2,3)

set.gamma.func.base <- function(S) {
  add.function <- function(y) {
    results <- apply(as.matrix(S_sub), 1, function(x) {
      return(sort(c(x, y)))
    })
    return(t(results))
  }

  set.gamma <- list()
  for(i in 1:3) {
    set.gamma[[i]] <- c()
  }

  # set of gamma-
  if(length(S) == 0) {
    S_sub <- (1:p)
    set.gamma[[1]] <- c()
    set.gamma[[2]] <- apply(as.matrix(S_sub), 1, function(x) { return(c(x, S)) })
    set.gamma[[2]] <- as.matrix(set.gamma[[2]])
    set.gamma[[3]] <- c()
  }
  if(length(S) > 1) {
    S_sub <- (1:p)[-S]
    set.gamma[[1]] <- t(combn(S, length(S) - 1))
    if(length(S) > 2) {
      set.gamma[[1]] <- t(apply(as.matrix(set.gamma[[1]]), 1, sort))
    }
    # set of gamma+
    set.gamma[[2]] <- apply(as.matrix(S_sub), 1, function(x) { return(sort(c(x, S))) })
    set.gamma[[2]] <- t(set.gamma[[2]])
    # set of gamma=
    set.gamma[[3]] <- add.function(set.gamma[[1]][1,])
    for(i in 2:nrow(set.gamma[[1]])) {
      set.gamma[[3]] <- rbind(set.gamma[[3]], add.function(set.gamma[[1]][i,]))
    }
  }
  if(length(S) == 1) {
    S_sub <- (1:p)[-S]
    set.gamma[[1]] <- t(combn(S, length(S) - 1))
    set.gamma[[2]] <- apply(as.matrix(S_sub), 1, function(x) { return(sort(c(x, S))) })
    set.gamma[[2]] <- t(set.gamma[[2]])
    set.gamma[[3]] <- t(add.function(set.gamma[[1]][1,]))
  }
  return(set.gamma)
}

result <- set.gamma.func.base(input_S)
print(result)


[[1]]
     [,1] [,2]
[1,]    1    2
[2,]    1    3
[3,]    2    3

[[2]]
     [,1] [,2] [,3] [,4]
[1,]    1    2    3    4
[2,]    1    2    3    5

[[3]]
     [,1] [,2] [,3]
[1,]    1    2    4
[2,]    1    2    5
[3,]    1    3    4
[4,]    1    3    5
[5,]    2    3    4
[6,]    2    3    5



In [8]:
set.gamma.func.conditional<-function(input.S,condition.index){
    add.function<-function(y){results<-(apply(as.matrix(S_sub),1,function(x){return(sort(c(x,y)))}))
    return(t(results))
    }
    
    set.gamma<-list()
    for(i in 1:3){
    set.gamma[[i]]<-c()
    }
    S=input.S[-match(condition.index,input.S)]
    
    #set of gamma-
    if(length(S)==0){
    S=integer(0)
    S_sub<-(1:p)[-condition.index]
    set.gamma[[1]]<-c()
    set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(c(x,S))})
    set.gamma[[2]]<-as.matrix(set.gamma[[2]])
    set.gamma[[3]]<-c()
    }
    if(length(S)==1){
    S_sub<-(1:p)[-input.S]
    set.gamma[[1]]<-t(combn(S,length(S)-1))
    set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
    set.gamma[[2]]<-as.matrix((t(set.gamma[[2]])))
    set.gamma[[3]]<-t(add.function(set.gamma[[1]][1,]))
    }
    if(length(S)>1){
    S_sub<-(1:p)[-input.S]
    if(length(S)>2){
        set.gamma[[1]]<-t(combn(S,length(S)-1))
        set.gamma[[1]]<-t(apply(as.matrix(set.gamma[[1]]),1,sort))
    }else{
        set.gamma[[1]]<-t(combn(S,length(S)-1))
    }
    set.gamma[[2]]<-apply(as.matrix(S_sub),1,function(x){return(sort(c(x,S)))})
    set.gamma[[2]]<-as.matrix(t(set.gamma[[2]]))
    set.gamma[[3]]<-add.function(set.gamma[[1]][1,])
    factorial(3)
    for(i in 2:nrow(set.gamma[[1]])){
        set.gamma[[3]]<-rbind(set.gamma[[3]],add.function(set.gamma[[1]][i,]))  
    }
    }
    
    return(set.gamma)
}

p <- 5
input_S <- c(1,2,3)
condition.index <- 2
result <- set.gamma.func.conditional(input_S, condition.index)
print(result)

[[1]]
     [,1]
[1,]    1
[2,]    3

[[2]]
     [,1] [,2] [,3]
[1,]    1    3    4
[2,]    1    3    5

[[3]]
     [,1] [,2]
[1,]    1    4
[2,]    1    5
[3,]    3    4
[4,]    3    5



In [26]:
#This function is made but not used 
duplicated.dgCMatrix <- function (dgCMat, MARGIN) {
  MARGIN <- as.integer(MARGIN)
  n <- nrow(dgCMat)
  p <- ncol(dgCMat)
  J <- rep(1:p, diff(dgCMat@p))
  I <- dgCMat@i + 1
  x <- dgCMat@x
  if (MARGIN == 1L) {
    ## check duplicated rows
    names(x) <- J
    RowLst <- split(x, I)
    is_empty <- setdiff(1:n, I)
    result <- duplicated.default(RowLst)
  } else if (MARGIN == 2L) {
    ## check duplicated columns
    names(x) <- I
    ColLst <- split(x, J)
    is_empty <- setdiff(1:p, J)
    result <- duplicated.default(ColLst)
  } else {
    warning("invalid MARGIN; return NULL")
    result <- NULL
  }
  
  if(any(is_empty)){
    out <- logical(if(MARGIN == 1L) n else p)
    out[-is_empty] <- result
    if(length(is_empty) > 1)
      out[is_empty[-1]] <- TRUE
    result <- out
  }
  
  result
}

library(Matrix)

# Create a sparse matrix with some duplicated rows and columns
mat <- matrix(c(1, 0, 1, 1, 0, 1, 1, 0, 1), nrow=3, byrow=TRUE)
dgCMat <- as(mat, "dgCMatrix")

# Check duplicated rows
result_rows <- duplicated.dgCMatrix(dgCMat, MARGIN=1)
print("Duplicated rows:")
print(result_rows)

# Check duplicated columns
result_cols <- duplicated.dgCMatrix(dgCMat, MARGIN=2)
print("Duplicated columns:")
print(result_cols)

# Test with invalid MARGIN value
result_invalid <- duplicated.dgCMatrix(dgCMat, MARGIN=3)
print("Result with invalid MARGIN:")
print(result_invalid)


[1] "Duplicated rows:"
[1] FALSE  TRUE  TRUE
[1] "Duplicated columns:"
[1] FALSE FALSE  TRUE


Warning message in duplicated.dgCMatrix(dgCMat, MARGIN = 3):
"invalid MARGIN; return NULL"


ERROR: Error in duplicated.dgCMatrix(dgCMat, MARGIN = 3): object 'is_empty' not found


In [27]:
match.dgCMatrix <- function (dgCMat1,dgCMat2) {
      #  dgCMat1=B.list[[2]]
      # dgCMat2=add.B[[2]]
      n1 <- nrow(dgCMat1)
      p1 <- ncol(dgCMat1)
      J1 <- rep(1:p1, diff(dgCMat1@p))
      I1 <- dgCMat1@i + 1
      x1 <- dgCMat1@x
      n2 <- nrow(dgCMat2)
      p2 <- ncol(dgCMat2)
      J2 <- rep(1:p2, diff(dgCMat2@p))
      I2 <- dgCMat2@i + 1
      x2 <- dgCMat2@x
      ## check duplicated rows
      names(x1) <- J1
      RowLst1 <- split(J1, I1)
      is_empty1<- setdiff(1:n1, I1)
      ## check duplicated rows
      names(x2) <- J2
      RowLst2 <- split(J2, I2)
      is_empty2 <- setdiff(1:n2, I2)
      result<-(match(RowLst2,RowLst1))
      if(any(which(result>is_empty1))){
        result[which(result>=is_empty1)]=result[which(result>=is_empty1)]+1
      }
      if(any(is_empty1)){
        if(any(is_empty2)){
          result<-c(is_empty1,result)  
        }
      }else{
        if(any(is_empty2)){
          result<-c(NA,result)  
        }
      }
      return(result)
      
    }


mat1 <- matrix(c(1, 0, 1, 1, 0, 0, 0, 1, 0), nrow=3, byrow=TRUE)
dgCMat1 <- as(mat1, "dgCMatrix")
mat2 <- matrix(c(1, 0, 1, 0, 1, 0, 1, 0, 1), nrow=3, byrow=TRUE)
dgCMat2 <- as(mat2, "dgCMatrix")

result <- match.dgCMatrix(dgCMat1, dgCMat2)
print("Matching result:")
print(result)


[1] "Matching result:"
[1] 1 3 1


In [33]:
PIP.func<-function(likeli,model.space){
    infi.index<-which(is.infinite(likeli))
    if(length(infi.index)!=0){
    likeli<-likeli[-infi.index]
    model.space<-model.space[-infi.index,]
    }
    na.index<-which(is.na(likeli))
    if(length(na.index)!=0){
    likeli<-likeli[-na.index]
    model.space<-model.space[-na.index,]
    }
    aa<-likeli-max(likeli,na.rm=T)
    prob.sum<-sum(exp(aa))
    result.prob<-rep(NA,p)
    for(i in 1:p){
    result.prob[i]<-sum(exp(aa[ which(model.space[,i]==1)]))/prob.sum
    }
    return(result.prob)
}

likeli <- c(log(0.4), log(0.5), Inf, log(0.3)) 
model.space <- matrix(c(1, 0, 1, 1, 0, 0, 1, 0), nrow = 4, byrow = TRUE) 
p <- ncol(model.space)

result <- PIP.func(likeli, model.space)
print(result)

[1] 1.0000000 0.4166667


In [4]:
'''
This code crashes in both ipynb and in RStuido
index.fun.inner<-function(x){
    m<-as(as(as(matrix(0,nrow=nrow(x),ncol=p), "dMatrix"), "generalMatrix"), "TsparseMatrix")
    m@i<-as.integer(rep(1:nrow(x)-1,each=ncol(x)))
    m@j<-as.integer(c(t(x))-1)
    m@x=rep(1,nrow(x)*ncol(x))
    m<-as(m,"CsparseMatrix")
    return(m)
}

x <- matrix(c(0, 1, 0, 1, 1, 0), nrow = 2)
p <- ncol(x)
result <- index.fun.inner(x)
print(result)
''''

: 

: 

In [4]:
index.fun.inner<-function(x){
    m<-as(as(as(matrix(0,nrow=nrow(x),ncol=p), "dMatrix"), "generalMatrix"), "TsparseMatrix")
    m@i<-as.integer(rep(1:nrow(x)-1,each=ncol(x)))
    m@j<-as.integer(c(t(x))-1)
    m@x=rep(1,nrow(x)*ncol(x))
    m<-as(m,"CsparseMatrix")
    return(m)
}

index.fun<-function(outer.x,Max.Model.Dimins=10){
    if(nrow(outer.x)>1000){
index.bins<-which((1:nrow(outer.x))%%floor(nrow(outer.x)/Max.Model.Dimins)==0)
result.m<-index.fun.inner(outer.x[1:index.bins[1],,drop=F])
for(b in 1:(length(index.bins)-1)){
    result.m<-rbind(result.m,index.fun.inner(outer.x[(index.bins[b]+1):index.bins[b+1],,drop=F]))
}
if(index.bins[length(index.bins)]!=nrow(outer.x)){
    result.m<-rbind(result.m,index.fun.inner(outer.x[(index.bins[length(index.bins)]+1):nrow(outer.x),,drop=F]))
}
    }else{
        result.m<-index.fun.inner(outer.x)
    }
return(result.m)
}

outer.x <- matrix(c(0, 1, 0, 1, 1, 0), nrow = 2)
p <- ncol(outer.x)

# Run the function
result <- index.fun(outer.x)
print(result)

: 

: 

In [22]:
index_vec_input <- c(1, 2)
Sigma <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
z <- c(1, 2)
tau <- 0.5
p_S <- 2
y_sigma <- 1

test.S.indices <- index_vec_input
modi.Sigma <- Sigma           
outlier.tau <- tau
modi.ld.S <- modi.Sigma[test.S.indices, test.S.indices]

outlier_likelihood <- outlier_Normal_fixed_sigma_marginal
ridge.fun <- function(x) {
  temp.ld.S <- x * modi.ld.S + (1 - x) * diag(nrow(modi.ld.S))
  temp.Sigma <- modi.Sigma
  temp.Sigma[test.S.indices, test.S.indices] <- temp.ld.S
  print(temp.ld.S)
  print(temp.Sigma)
  print(test.S.indices)
  return(outlier_likelihood(test.S.indices, temp.Sigma, z, outlier.tau, length(test.S.indices), 1))
}

x <- 0.5
result <- ridge.fun(x)
print(result)


     [,1] [,2]
[1,] 1.00 0.25
[2,] 0.25 1.00
     [,1] [,2]
[1,] 1.00 0.25
[2,] 0.25 1.00
[1] 1 2
[1] -0.6788419
